In [53]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.cluster import KMeans

In [54]:
userdf = pd.read_csv(r"D:\Courses\GP-Cinemate\ml-1m\ml-1m\users.dat", delimiter="::", header=None, engine='python')

In [55]:
userdf.head(-1)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706


In [56]:
userdf = userdf.drop([3, 4], axis=1) 
userdf[1] = userdf[1].replace({'M': 1, 'F': 0})
userdf.columns = ["UserID", "Gender", "Age"]

C:\Users\Ziad\AppData\Local\Temp\ipykernel_20156\1457730755.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  userdf[1] = userdf[1].replace({'M': 1, 'F': 0})


In [57]:
userdf.info()
userdf.head(-1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   UserID  6040 non-null   int64
 1   Gender  6040 non-null   int64
 2   Age     6040 non-null   int64
dtypes: int64(3)
memory usage: 141.7 KB


,UserID,Gender,Age
0,1,0,1
1,2,1,56
2,3,1,25
3,4,1,45
4,5,1,25
...,...,...,...
6034,6035,0,25
6035,6036,0,25
6036,6037,0,45
6037,6038,0,56


In [58]:
moviedf = pd.read_csv(r"D:\Courses\GP-Cinemate\ml-1m\ml-1m\movies.dat", delimiter="::", header=None, encoding='latin-1', engine='python')

In [59]:
moviedf.head(-1)

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3877,3947,Get Carter (1971),Thriller
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama


In [60]:
moviedf.columns = ["MovieID", "Title", "Genres"]
moviedf['Year'] = moviedf['Title'].str.extract(r'\((\d{4})\)')
moviedf['Title'] = moviedf['Title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()
moviedf['Genres'] = moviedf['Genres'].str.split('|')
moviedf.head(-1)

,MovieID,Title,Genres,Year
0,1,Toy Story,"[Animation, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
3877,3947,Get Carter,[Thriller],1971
3878,3948,Meet the Parents,[Comedy],2000
3879,3949,Requiem for a Dream,[Drama],2000
3880,3950,Tigerland,[Drama],2000


In [61]:
genre_mapping = {
    "Action": 1, "Adventure": 2, "Animation": 3, "Children's": 4, "Comedy": 5,
    "Crime": 6, "Documentary": 7, "Drama": 8, "Fantasy": 9, "Film-Noir": 10,
    "Horror": 11, "Musical": 12, "Mystery": 13, "Romance": 14, "Sci-Fi": 15,
    "Thriller": 16, "War": 17, "Western": 18
}

In [62]:
reverse_genre_mapping = {v: k for k, v in genre_mapping.items()}

In [63]:
moviedf['Genres'] = moviedf['Genres'].apply(lambda genres: [genre_mapping[g] for g in genres])

In [64]:
ratingdf = pd.read_csv(r"D:\Courses\GP-Cinemate\ml-1m\ml-1m\ratings.dat", delimiter="::", header=None, engine='python')

In [65]:
ratingdf.info()
ratingdf.head(-1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   0       1000209 non-null  int64
 1   1       1000209 non-null  int64
 2   2       1000209 non-null  int64
 3   3       1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746


In [66]:
ratingdf = ratingdf.drop([3], axis=1)
ratingdf.columns = ["UserID", "MovieID", "Rating"]
ratingdf.head(-1)

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5


In [67]:
ratingdf.head(-1)

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5


In [68]:
ratingdf['Rating'].value_counts()
ratingdf['Rating'].describe()

count    1.000209e+06
mean     3.581564e+00
std      1.117102e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: Rating, dtype: float64

In [69]:
merged_df = ratingdf.merge(userdf, on="UserID", how="inner")
merged_df = merged_df.merge(moviedf, on="MovieID", how="inner")
merged_df.to_csv(r"D:\Courses\GP-Cinemate\ML\machine-learning-dev\merged_data.csv", index=False)
print(merged_df.head())

   UserID  MovieID  Rating  Gender  Age                            Title  \
0       1     1193       5       0    1  One Flew Over the Cuckoo's Nest   
1       1      661       3       0    1        James and the Giant Peach   
2       1      914       3       0    1                     My Fair Lady   
3       1     3408       4       0    1                  Erin Brockovich   
4       1     2355       5       0    1                    Bug's Life, A   

       Genres  Year  
0         [8]  1975  
1  [3, 4, 12]  1996  
2    [12, 14]  1964  
3         [8]  2000  
4   [3, 4, 5]  1998  


In [70]:
age_group_mapping = {
    1: "Under 18",
    18: "18-24",
    25: "25-34",
    35: "35-44",
    45: "45-49",
    50: "50-55",
    56: "56+"
}

In [71]:
def assign_age_group(age):
    if age == 1:
        return "Under 18"
    elif age <= 24:
        return "18-24"
    elif age <= 34:
        return "25-34"
    elif age <= 44:
        return "35-44"
    elif age <= 49:
        return "45-49"
    elif age <= 55:
        return "50-55"
    else:
        return "56+"

In [72]:
merged_df['AgeGroup'] = merged_df['Age'].apply(assign_age_group)

In [73]:
def map_genres(genre_list):
    return [reverse_genre_mapping.get(g, "Unknown") for g in genre_list]

In [74]:
top_movies_by_age_group = {}
for age_group in merged_df['AgeGroup'].unique():
    age_group_df = merged_df[merged_df['AgeGroup'] == age_group].copy()
    
    movie_ratings = age_group_df.groupby(['MovieID', 'Title']).agg({
        'Rating': ['mean', 'count'],
        'Genres': 'first',  
        'Year': 'first'
    }).reset_index()
    
    movie_ratings.columns = ['MovieID', 'Title', 'AvgRating', 'RatingCount', 'Genres', 'Year']
    
    min_ratings = 5
    if len(movie_ratings[movie_ratings['RatingCount'] >= min_ratings]) >= 10:
        filtered_movies = movie_ratings[movie_ratings['RatingCount'] >= min_ratings]
    else:
        filtered_movies = movie_ratings
    
    top_10 = filtered_movies.sort_values(
        by=['AvgRating', 'RatingCount'], 
        ascending=[False, False]
    ).head(10)
    
    top_movies_by_age_group[age_group] = top_10

In [75]:
final_results = []
for age_group, df in top_movies_by_age_group.items():
    df['AgeGroup'] = age_group
    
    df['GenresText'] = df['Genres'].apply(map_genres)
    
    final_results.append(df)

In [76]:
final_df = pd.concat(final_results, ignore_index=True)

In [77]:
output_df = final_df[['AgeGroup', 'MovieID', 'Title', 'GenresText', 'AvgRating', 'RatingCount', 'Year']]

In [78]:
output_df.to_csv(r"D:\Courses\GP-Cinemate\ML\machine-learning-dev\preprocessing\top_movies_by_age_group.csv", index=False)

print("\nSample of top movies by age group:")
for age_group in sorted(top_movies_by_age_group.keys()):
    movies = top_movies_by_age_group[age_group]['Title'].tolist()
    ratings = top_movies_by_age_group[age_group]['AvgRating'].tolist()
    counts = top_movies_by_age_group[age_group]['RatingCount'].tolist()

    print(f"\nAge Group: {age_group}")
    print("Top 3 movies (Title, Avg Rating, # of Ratings):")
    for i in range(min(3, len(movies))):
        print(f"  {movies[i]} - {ratings[i]:.2f} ({counts[i]} ratings)")

PermissionError: [Errno 13] Permission denied: 'D:\\Courses\\GP-Cinemate\\ML\\machine-learning-dev\\preprocessing\\top_movies_by_age_group.csv'